### Install Packages

In [14]:
#pip install datasets[audio] transformers accelerate evaluate jiwer

### Load Training Datasets

In [15]:
from datasets import load_dataset

data = load_dataset("trainData/audio", data_files="../asr.jsonl", split="train")
data = data.train_test_split()

### Prepare Processor and Dataset

In [16]:
from transformers import WhisperProcessor
import librosa

processor = WhisperProcessor.from_pretrained("openai/whisper-base.en")

def prepare_dataset(batch):
    # load audio data
    audio_data, sampling_rate = librosa.load("trainData/audio/"+batch["audio"], sr=None) 

    # compute input features 
    batch["input_features"] = processor.feature_extractor(audio_data, sampling_rate=sampling_rate).input_features[0]

    # encode target text to label ids 
    batch["labels"] = processor.tokenizer(batch["transcript"]).input_ids
    return batch

prep_data = data.map(prepare_dataset, remove_columns=data.column_names["train"])
print(prep_data)


Map: 100%|██████████| 3/3 [00:00<00:00, 69.73 examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 8
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 3
    })
})


### Evaluation Metrics

In [17]:
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Data Collator

In [20]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

### Load Pretrained Model and Initialise Data Collator

In [21]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base.en")
model.generation_config.forced_decoder_ids = None

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

### Training Arguments and Trainer

In [23]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./model8TRAIN",  # change to a repo name of your choice
    learning_rate=1e-5,
    warmup_steps=5,
    max_steps=50,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=10,
    eval_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
)

In [25]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=prep_data["train"],
    eval_dataset=prep_data["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs
  0%|          | 0/50 [00:00<?, ?it/s]c:\Users\Wan Sim\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
                                               
 20%|██        | 10/50 [03:49<15:12, 22.80s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strat

{'eval_loss': 0.20561376214027405, 'eval_wer': 7.547169811320755, 'eval_runtime': 5.3832, 'eval_samples_per_second': 0.557, 'eval_steps_per_second': 0.186, 'epoch': 10.0}


c:\Users\Wan Sim\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
                                               
 40%|████      | 20/50 [07:36<11:04, 22.16s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 

{'eval_loss': 0.11145203560590744, 'eval_wer': 3.7735849056603774, 'eval_runtime': 5.4671, 'eval_samples_per_second': 0.549, 'eval_steps_per_second': 0.183, 'epoch': 20.0}


c:\Users\Wan Sim\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
                                               
 60%|██████    | 30/50 [11:16<07:10, 21.52s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 

...

c:\Users\Wan Sim\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
                                               
100%|██████████| 50/50 [18:59<00:00, 23.52s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 

{'eval_loss': 0.10304304957389832, 'eval_wer': 3.7735849056603774, 'eval_runtime': 6.3421, 'eval_samples_per_second': 0.473, 'eval_steps_per_second': 0.158, 'epoch': 50.0}


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].
100%|██████████| 50/50 [19:00<00:00, 22.81s/it]

{'train_runtime': 1140.5127, 'train_samples_per_second': 0.351, 'train_steps_per_second': 0.044, 'train_loss': 0.09697512626647949, 'epoch': 50.0}


TrainOutput(global_step=50, training_loss=0.09697512626647949, metrics={'train_runtime': 1140.5127, 'train_samples_per_second': 0.351, 'train_steps_per_second': 0.044, 'total_flos': 2.5943998464e+16, 'train_loss': 0.09697512626647949, 'epoch': 50.0})

### Saving Model

In [26]:
trainer.save_model("asrModel8")
WhisperProcessor.save_pretrained(processor, "asrModel8proc")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 357, 366, 438, 532, 685, 705, 796, 930, 1058, 1220, 1267, 1279, 1303, 1343, 1377, 1391, 1635, 1782, 1875, 2162, 2361, 2488, 3467, 4008, 4211, 4600, 4808, 5299, 5855, 6329, 7203, 9609, 9959, 10563, 10786, 11420, 11709, 11907, 13163, 13697, 13700, 14808, 15306, 16410, 16791, 17992, 19203, 19510, 20724, 22305, 22935, 27007, 30109, 30420, 33409, 34949, 40283, 40493, 40549, 47282, 49146, 50257, 50357, 50358, 50359, 50360, 50361], 'begin_suppress_tokens': [220, 50256]}


[]

### Testing

In [27]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
from pathlib import Path

model_path = Path(f"asrModel8")
proc_path = Path(f"asrModel8proc")

model = WhisperForConditionalGeneration.from_pretrained(model_path)
processor = WhisperProcessor.from_pretrained(proc_path)

test = load_dataset("testData/audio")
print(test)
for i in range(10):
    input_speech = test["train"][i]["audio"]
    input_features = processor(input_speech["array"], sampling_rate=input_speech["sampling_rate"], return_tensors="pt").input_features
    predicted_ids = model.generate(input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    print(transcription)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


DatasetDict({
    train: Dataset({
        features: ['audio'],
        num_rows: 10
    })
})
[' Taurus, prepare to deploy electromagnetic pulse, heading 065, target is grey, and white fighter jets, engage when ready.']
[' Engage yellow drone with surface-to-air missiles heading two three five.']
[' Tarrat Bravo, engage the target, white, green, and silver cargo aircraft, heading zero five five with surface-to-air missiles, prepare for launch.']
[' Control tower to turrets, deploy EMPA towards green, black, and silver missile, heading one five zero.']
[' turrets, engage targets, black and red cargo aircraft, heading two zero zero, deploy EMP.']
[' Controlled to turrets, we have a purple light aircraft at heading three four zero, deploy electromagnetic pulse, stay alert, and be prepared for immediate action.']
[' Engage red and blue cargo aircraft at heading one one zero with surface-to-air missiles.']
[' Torrebravo, engaged silver, red, and yellow missiles, adheading two three five wi